ЗАДАНИЕ 2

In [1]:
import sqlite3

connection = sqlite3.connect("датасеты_к_заданию_2/Chinook_Sqlite.sqlite")

In [9]:
cur = connection.cursor()

In [14]:
def findAlbumForArtist(name_artist):
    first_q = """Select ArtistId From Artist WHERE Name = ?"""
    id_artist = cur.execute(first_q, [name_artist]).fetchall()
    id_artist =  id_artist[0][0]
    q_itog = 'Select Title FROM Album WHERE ArtistId = ?'
    res = cur.execute(q_itog, [id_artist]).fetchall()
    return res
    

In [15]:
findAlbumForArtist("AC/DC")

[('For Those About To Rock We Salute You',), ('Let There Be Rock',)]

In [17]:
def getCountTrackAndSummaryTimeForArtist(name_artist, name_album):
    first_q = "Select ArtistId From Artist WHERE Name = ?"
    id_artist = cur.execute(first_q, [name_artist]).fetchall()
    
    id_artist =  id_artist[0][0]
    q_itog = 'Select AlbumId FROM Album WHERE Title = ? AND ArtistId = ?'
    album_id = cur.execute(q_itog, [name_album, id_artist]).fetchall()
    album_id = album_id[0][0]
    
    q_count_tracks = 'Select COUNT(Name) FROM Track WHERE AlbumId = ?'
    count_tracks = cur.execute(q_count_tracks, [album_id]).fetchall()
    
    q_sum_len = 'Select SUM(Milliseconds) FROM Track WHERE AlbumId = ?'
    len_tracks = cur.execute(q_sum_len, [album_id]).fetchall()
    
    return count_tracks[0][0], len_tracks[0][0]

In [22]:
count, len_tracks = getCountTrackAndSummaryTimeForArtist("AC/DC", "Let There Be Rock")

count, len_tracks

(8, 2453259)

ЗАДАНИЕ 3

In [58]:
import dask as dd
from glob import glob
import pandas as pd
from dask import compute, delayed
from collections import defaultdict, OrderedDict
from pprint import pprint

In [59]:
filenames = sorted(glob("датасеты_к_заданию_3/data/*.csv"))

filenames

['датасеты_к_заданию_3/data\\accounts.0.csv',
 'датасеты_к_заданию_3/data\\accounts.1.csv',
 'датасеты_к_заданию_3/data\\accounts.2.csv',
 'датасеты_к_заданию_3/data\\accounts.3.csv',
 'датасеты_к_заданию_3/data\\accounts.4.csv',
 'датасеты_к_заданию_3/data\\accounts.5.csv']

In [60]:
sums = []

for x in filenames:
    df = delayed(pd.read_csv)(x)
    t = df[(df.amount > 999) & (df.amount < 1501)][["id","amount"]].groupby(df.id).size()
    sums.append(t)

In [61]:
sums = compute(sums)
sums

([id
  2         6
  3         1
  4      2282
  10      912
  15     4120
         ... 
  487     291
  492     151
  494      56
  496       3
  497      16
  Length: 121, dtype: int64,
  id
  2          7
  3          2
  4       2099
  10      1001
  15      4035
         ...  
  485    11373
  487      299
  492      138
  494       50
  497       16
  Length: 121, dtype: int64,
  id
  2          9
  3          1
  4       2214
  10       954
  15      3994
         ...  
  485    11156
  487      294
  492      132
  494       42
  497       12
  Length: 121, dtype: int64,
  id
  2          3
  3          1
  4       2155
  10       999
  15      4053
         ...  
  485    11390
  487      312
  492      115
  494       47
  497       21
  Length: 118, dtype: int64,
  id
  2          5
  3          1
  4       2124
  10       992
  15      3929
         ...  
  485    11337
  487      311
  492      135
  494       50
  497       17
  Length: 119, dtype: int64,
  id
  2        

In [63]:
final = defaultdict(lambda: 0)

for df in sums[0]:
    for k, v in df.to_dict().items():
        final[k] = final[k] + v
print(sorted(final.items(), key=lambda x: x[1], reverse=True)[0][0])

485
